---

# Lab 3 – Deep Learning for NLP
Master MSITBD – FST Tanger  
Professor: Pr. ELAACHAK LOTFI  
Student: ABABRI Chaimae


## ==========================================

## Part1 Classification Task

In [16]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from pyarabic.araby import strip_tashkeel


/usr/local/lib/python3.12/dist-packages/pyarabic/araby.py:274: SyntaxWarning: invalid escape sequence '\w'
  TOKEN_PATTERN = re.compile(u"([^\w\u0670\u064b-\u0652']+)", re.UNICODE)
/usr/local/lib/python3.12/dist-packages/pyarabic/araby.py:276: SyntaxWarning: invalid escape sequence '\w'
  TOKEN_PATTERN_SPLIT = re.compile(u"([\w\u0670\u064b-\u0652']+)", re.UNICODE)
/usr/local/lib/python3.12/dist-packages/pyarabic/araby.py:281: SyntaxWarning: invalid escape sequence '\s'
  ARABIC_STRING = re.compile(u"([^\u0600-\u0652%s%s%s\s\d])" \
/usr/local/lib/python3.12/dist-packages/pyarabic/araby.py:1237: SyntaxWarning: invalid escape sequence '\s'
  u"(?<=\s(%s|%s))%s" % (WAW, YEH, FATHA), \
/usr/local/lib/python3.12/dist-packages/pyarabic/araby.py:1450: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub(u"(?<=[\s\d])([%s])+"%(TASHKEEL_STRING),"",text,  re.UNICODE)


In [17]:

nltk.download('stopwords')
stop_words = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
len(stop_words)

701

In [86]:
import requests
from bs4 import BeautifulSoup
import time
import random

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def get_article_links(url, link_selector, base_domain):
    try:
        html = requests.get(url, headers=headers, timeout=10).text
        soup = BeautifulSoup(html, "html.parser")
        links = []
        for a in soup.select(link_selector):
            href = a.get('href')
            if href:
                if href.startswith('/'):
                    href = base_domain + href
                elif not href.startswith('http'):
                    continue
                links.append(href)
        return list(set(links))  #supprimer les doublons
    except Exception as e:
        print(f"Erreur lors de la récupération des liens : {  (e)}")
        return []

def scrape_article_text(url, text_selector):
    try:
        html = requests.get(url, headers=headers, timeout=10).text
        soup = BeautifulSoup(html, "html.parser")
        texts = []
        for p in soup.select(text_selector):
            text = p.text.strip()
            if len(text) > 50:
                texts.append(text)
        return texts
    except Exception as e:
        print(f"Erreur sur l'article {url} : {e}")
        return []

# Sites fiables + accessibles
sites = {
    "Aljazeera": {
        "list_url": "https://www.aljazeera.net/tech/",
        "link_selector": "article a[href^='/tech/'], article a[href*='/2025/']",
        "base_domain": "https://www.aljazeera.net",
        "text_selector": "div.article-body p, .wysiwyg p"
    },
    "Alarabiya": {
        "list_url": "https://www.alarabiya.net/technology",
        "link_selector": "a[href^='/technology/'][href*='/2025/'], h3 a[href]", 
        "base_domain": "https://www.alarabiya.net",
        "text_selector": ".article-body p, p"
    }
}

all_texts = []
max_articles_per_site = 10  # pour eviter le blocage

for name, config in sites.items():
    print(f"Scraping en cours de {name}...")
    links = get_article_links(config["list_url"], config["link_selector"], config["base_domain"])
    links = links[:max_articles_per_site]  # limiter
    print(f"Liens d'articles trouvés : {len(links)}")
    
    for link in links:
        print(f"Extraction en cours : {link}")
        texts = scrape_article_text(link, config["text_selector"])
        all_texts.extend(texts)
        time.sleep(random.uniform(2, 5))  # delai al2atoire pour eviter la detection

print("Total de paragraphes collectés :", len(all_texts))

Scraping en cours de Aljazeera...
Liens d'articles trouvés : 10
Extraction en cours : https://www.aljazeera.net/tech/2025/12/16/%d8%a3%d9%8a%d9%86-%d8%aa%d8%b0%d9%87%d8%a8-%d8%b3%d8%ac%d9%84%d8%a7%d8%aa-%d8%b4%d8%a7%d8%aa-%d8%ac%d9%8a-%d8%a8%d9%8a-%d8%aa%d9%8a-%d8%b9%d9%86%d8%af%d9%85%d8%a7-%d9%8a%d9%85%d9%88%d8%aa
Extraction en cours : https://www.aljazeera.net/ebusiness/2025/12/17/%d9%87%d9%84-%d8%a7%d9%86%d8%aa%d9%87%d9%89-%d8%b9%d8%b5%d8%b1-%d8%a7%d9%84%d9%88%d8%b8%d8%a7%d8%a6%d9%81-%d8%a7%d9%84%d8%b0%d9%83%d8%a7%d8%a1
Extraction en cours : https://www.aljazeera.net/ebusiness/2025/12/14/%d9%83%d9%88%d8%b1%d9%8a%d8%a7-%d8%aa%d9%83%d9%86%d9%88%d9%84%d9%88%d8%ac%d9%8a%d8%a7-%d8%a7%d8%aa%d8%b5%d8%a7%d9%84%d8%a7%d8%aa-%d8%b5%d8%b9%d9%88%d8%af-%d8%b5%d8%a7%d8%af%d8%b1%d8%a7%d8%aa
Extraction en cours : https://www.aljazeera.net/ebusiness/2025/12/13/%d8%a3%d9%8a%d9%86-%d8%aa%d8%b3%d8%aa%d8%ab%d9%85%d8%b1-%d9%81%d9%8a-%d8%a7%d9%84%d8%b0%d9%83%d8%a7%d8%a1-%d8%a7%d9%84%d8%a7%d8%b5%d8%b7%d9%86

In [87]:
all_texts

['رفضت شركة "أوبن إيه آي" مشاركة سجل المحادثات الكامل المرتبط بقضية القتل التي أثارت الرأي العام الأميركي في الأسابيع الماضية، إذ قتل شتاين-إريك سولبيرغ والدته البالغة من العمر 83 عاما بعد محادثات مطولة مع "شات جي بي تي"، وذلك وفق تقرير لموقع "آرس تكنيكا" التقني.',
 'وتعالت الانتقادات الموجهة إلى "أوبن إيه آي" بعد رفضها مشاركة سجلات المحادثة هذه، خاصة وأن أسرة الضحية سوزان آدمز اتهمت الشركة رسميا بإخفاء سجلات المحادثة عمدا لتبرئة نفسها والنموذج الخاص بها، وذلك في القضية المقدمة أمام المحكمة العليا لولاية كاليفورنيا.',
 'وبفضل مجموعة من مقاطع الفيديو ومنشورات التواصل الاجتماعي التي كان سولبيرغ يشاركها عبر حساباته، تمكنت الأسرة من الوصول إلى أجزاء من محادثاته مع "شات جي بي تي".',
 'وأظهرت هذه المحادثات أن "شات جي بي تي" ساهم في تغذية أوهام العظمة التي كان يعاني منها سولبيرغ، كما وضع والدته في منتصف هذه الأوهام وجعلها تبدو كالعدو الرئيسي في روايته.',
 'ويؤكد إريك سولبيرغ حفيد الضحية سوزان آدمز وابن الجاني في بيان رسمي، أن "أوبن إيه آي" اختارت عمدا إخفاء سجلات المحادثة للأيام والأسابيع الأ

In [105]:
keywords = ["تكنولوجيا", "ذكاء اصطناعي", "AI", "روبوت", "تقنية"]

def compute_score(text):
    count = sum(text.count(k) for k in keywords)
    
    if count == 0:
        return 2
    elif count == 1:
        return 4
    elif count == 2:
        return 6
    elif count >= 3:
        return 8


In [106]:
def preprocess(text):
    text = strip_tashkeel(text)
    text = re.sub(r"[^\u0600-\u06FF ]", "", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

texts = all_texts  # utiliser la liste scrapp2e
processed_texts = [preprocess(t) for t in texts]

In [107]:

from collections import Counter

counter = Counter(" ".join(processed_texts).split())
vocab = {word: i+1 for i, (word, _) in enumerate(counter.items())}
vocab["<pad>"] = 0

def encode(text):
    return [vocab[word] for word in text.split() if word in vocab]

In [108]:
class TextDataset(Dataset):
    def __init__(self, texts, scores):
        self.texts = texts
        self.scores = scores

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        x = torch.tensor(encode(self.texts[idx]), dtype=torch.long)
        y = torch.tensor(self.scores[idx], dtype=torch.float)
        return x, y


In [109]:

def collate_fn(batch):
    xs, ys = zip(*batch)
    xs = nn.utils.rnn.pad_sequence(xs, batch_first=True)
    ys = torch.tensor(ys)
    return xs, ys

X_train, X_test, y_train, y_test = train_test_split(
    processed_texts, scores, test_size=0.3, random_state=42
)

train_ds = TextDataset(X_train, y_train)
test_ds = TextDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=4, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=4, collate_fn=collate_fn)

In [110]:

class NLPModel(nn.Module):
    def __init__(self, vocab_size, model_type="RNN"):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, 128)

        if model_type == "RNN":
            self.rnn = nn.RNN(128, 64, batch_first=True)
            self.fc = nn.Linear(64, 1)

        elif model_type == "BiRNN":
            self.rnn = nn.RNN(128, 64, bidirectional=True, batch_first=True)
            self.fc = nn.Linear(128, 1)

        elif model_type == "GRU":
            self.rnn = nn.GRU(128, 64, batch_first=True)
            self.fc = nn.Linear(64, 1)

        elif model_type == "LSTM":
            self.rnn = nn.LSTM(128, 64, batch_first=True)
            self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = self.embed(x)
        out, h = self.rnn(x)
        
        if isinstance(h, tuple):  # LSTM
            h = h[0]  # prendre h_n
         
        if self.rnn.bidirectional:
            # concatener les deux directions
            h = torch.cat([h[-2], h[-1]], dim=1)  # shape = (batch, 128)
        else:
            h = h[-1]  
        
        return self.fc(h).squeeze(1)


In [111]:

def train_and_evaluate(model_type):
    model = NLPModel(len(vocab), model_type)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    for epoch in range(10):
        for x, y in train_loader:
            optimizer.zero_grad()
            pred = model(x)
            y = y.float()
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()

    preds, refs = [], []
    for x, y in test_loader:
        p = model(x)
        preds.extend(p.detach().numpy())
        refs.extend(y.numpy())

    mse = mean_squared_error(refs, preds)
    mae = mean_absolute_error(refs, preds)
    bleu = sentence_bleu([str(refs)], str(preds))

    print(f"{model_type} → MSE: {mse:.3f}, MAE: {mae:.3f}, BLEU: {bleu:.3f}")

In [112]:

for model in ["RNN", "BiRNN", "GRU", "LSTM"]:
    train_and_evaluate(model)

RNN → MSE: 0.183, MAE: 0.389, BLEU: 0.171
BiRNN → MSE: 0.194, MAE: 0.353, BLEU: 0.162
GRU → MSE: 0.189, MAE: 0.365, BLEU: 0.182
LSTM → MSE: 0.175, MAE: 0.345, BLEU: 0.186


---

## Part 2 – Transformer (GPT-2)

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
model_name = "gpt2"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

print(f" GPT-2 chargé : {model.num_parameters():,} paramètres")


2025-12-18 23:13:34.972330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766099615.154240      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766099615.207392      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766099615.646021      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766099615.646064      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766099615.646066      55 computation_placer.cc:177] computation placer alr

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

 GPT-2 chargé : 124,439,808 paramètres


In [3]:
custom_texts = [
    "Artificial intelligence is transforming modern technology.",
    "Deep learning relies on neural networks.",
    "Natural language processing helps computers understand text.",
    "Transformers are powerful models for text generation."
]


print("Nombre de textes :", len(custom_texts))


Nombre de textes : 4


In [4]:
encodings = tokenizer(
    custom_texts,
    return_tensors="pt",
    padding=True,
    truncation=True
)

input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]

print("Shape input_ids :", input_ids.shape)


Shape input_ids : torch.Size([4, 9])


In [5]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.train()

epochs = 3

for epoch in range(epochs):
    optimizer.zero_grad()

    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=input_ids
    )

    loss = outputs.loss
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}/{epochs} — Loss = {loss.item():.4f}")

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch 1/3 — Loss = 5.2193
Epoch 2/3 — Loss = 4.2808
Epoch 3/3 — Loss = 3.7163


In [6]:
model.eval()

prompt = "Artificial intelligence"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

generated_ids = model.generate(
    input_ids,
    max_length=60,
    do_sample=True,
    top_k=50,
    top_p=0.95
)

generated_text = tokenizer.decode(
    generated_ids[0],
    skip_special_tokens=True
)

print(" Texte généré :")
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Texte généré :
Artificial intelligence, which is described as a type of artificial intelligence, is widely used in the aerospace industry.

Advertisement

If it becomes sufficiently powerful, robots could be used for anything from building aircraft to diagnosing people. Even so, the future of artificial intelligence could be much less ro
